<a href="https://colab.research.google.com/github/ma850419/Various_scripts/blob/main/from_gee_to_colab_archeology_26May2026.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import ee
ee.Authenticate()
ee.Initialize(project='velvety-ring-328419')

In [ ]:
# Install geemap if not already installed
!pip install geemap



In [ ]:
import geemap
# Define region of interest
m = geemap.Map(center=[-14.2, -51.9], zoom=4)
mask = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017').filter(ee.Filter.eq('country_na', 'Brazil'))
SA = mask.geometry()

# Load archaeological points
archaeology_points = ee.FeatureCollection('users/mohamadawadlebanon/Archeologicalsites')

# Define date range
start_date = '2024-04-01'
end_date = '2024-04-02'

### Step 1: Mosaic Sentinel-1 images per date ###
def mosaic_sentinel1(date):
    date = ee.Date(date)
    sentinel1_images = ee.ImageCollection("COPERNICUS/S1_GRD").filterBounds(SA) \
        .filterDate(date, date.advance(1, 'day')) \
        .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')) \
        .filter(ee.Filter.eq('instrumentMode', 'IW')) \
        .select("VV")

    return sentinel1_images.median().set("date", date.format("YYYY-MM-dd"))

### Step 2: Mosaic Sentinel-2 images per date ###
def mosaic_sentinel2(date):
    date = ee.Date(date)
    sentinel2_images = ee.ImageCollection("COPERNICUS/S2").filterBounds(SA) \
        .filterDate(date, date.advance(1, 'day')) \
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 90)) \
        .select(["B8", "B11", "B12"])

    return sentinel2_images.median().set("date", date.format("YYYY-MM-dd"))

# Extract available dates
sentinel1_dates = ee.ImageCollection("COPERNICUS/S1_GRD").filterBounds(SA) \
    .filterDate(start_date, end_date) \
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')) \
    .filter(ee.Filter.eq('instrumentMode', 'IW')) \
    .aggregate_array('system:time_start').map(ee.Date)

sentinel2_dates = ee.ImageCollection("COPERNICUS/S2").filterBounds(SA) \
    .filterDate(start_date, end_date) \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10)) \
    .aggregate_array('system:time_start').map(ee.Date)

# Find matching dates between Sentinel-1 and Sentinel-2
formatted_sentinel1_dates = sentinel1_dates.map(lambda date: ee.Date(date).format("YYYY-MM-dd"))
formatted_sentinel2_dates = sentinel2_dates.map(lambda date: ee.Date(date).format("YYYY-MM-dd"))

common_dates = formatted_sentinel1_dates.filter(ee.Filter.inList('item', formatted_sentinel2_dates))

# Create mosaiced collections
mosaiced_sentinel1 = ee.ImageCollection(common_dates.map(mosaic_sentinel1))
mosaiced_sentinel2 = ee.ImageCollection(common_dates.map(mosaic_sentinel2))

### Step 3: Combine Sentinel-1 & Sentinel-2 mosaiced images and classify ###
def classify_image(date):
    date = ee.Date(date)

    sentinel2 = mosaiced_sentinel2.filter(ee.Filter.eq("date", date.format("YYYY-MM-dd"))).first()
    sentinel1 = mosaiced_sentinel1.filter(ee.Filter.eq("date", date.format("YYYY-MM-dd"))).first()

    combined = sentinel2.addBands(sentinel1)

    classified = ee.Algorithms.If(
        archaeology_points.size().gt(0),
        combined.classify(
            ee.Classifier.smileRandomForest(10).train(
                combined.sampleRegions(collection=archaeology_points, properties=['class'], scale=30),
                'class'
            )
        ),
        ee.Image.constant(-9999).rename("classification")  # Placeholder if no training points
    )

    return ee.Image(classified).set("date", date.format("YYYY-MM-dd"))


# Create Map
m = geemap.Map(center=[-14.2, -51.9], zoom=4)

# Display Sentinel-1 mosaiced images
list_s1 = mosaiced_sentinel1.toList(mosaiced_sentinel1.size())
for i in range(list_s1.size().getInfo()):
    img = ee.Image(list_s1.get(i))
    date_label = img.get("date").getInfo()
    vis_params_s1 = {"bands": ["VV"], "min": -20, "max": 0, "gamma": 1.4}
    m.addLayer(img, vis_params_s1, f"Sentinel-1 Mosaiced ({date_label})")

# Display Sentinel-2 mosaiced images
list_s2 = mosaiced_sentinel2.toList(mosaiced_sentinel2.size())
for i in range(list_s2.size().getInfo()):
    img = ee.Image(list_s2.get(i))
    date_label = img.get("date").getInfo()
    vis_params_s2 = {"bands": ["B12", "B11", "B8"], "min": 0, "max": 3000, "gamma": 1.4}
    m.addLayer(img, vis_params_s2, f"Sentinel-2 Mosaiced ({date_label})")
# Add the archaeology points layer
archaeology_vis = {
    'color': 'blue',  # Color of the points
    'pointRadius': 5  # Size of the points (in pixels)
}
m.addLayer(archaeology_points, archaeology_vis, 'Archaeology Points')
# Apply classification
'''classified_images = ee.ImageCollection(common_dates.map(classify_image))
print("Number of classified images:", classified_images.size().getInfo())
# Display the map
# Display Classified Images
list_classified = classified_images.toList(classified_images.size())
for i in range(list_classified.size().getInfo()):
    img = ee.Image(list_classified.get(i))
    date_label = img.get("date").getInfo()
    vis_params_classified = {
        "bands": ["classification"],
        "min": 0,
        "max": 1,
        "palette": ["red", "yellow", "green"]
    }
    m.addLayer(img, vis_params_classified, f"Classified Archaeological Predictions ({date_label})")'''

m.addLayer(mask, {"color": "black"}, "South America Mask")
m



In [ ]:
import geemap

# Define region of interest
m = geemap.Map(center=[-14.2, -51.9], zoom=4)
mask = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017').filter(ee.Filter.eq('country_na', 'Venezuela'))
SA = mask.geometry()
m.addLayer(mask, {"color": "black"}, "South America Mask")

# Load archaeological points
archaeology_points = ee.FeatureCollection('users/mohamadawadlebanon/Archeologicalsites')

# Define date range
start_date = '2024-04-01'
end_date = '2024-04-30'

### Step 1: Stitch Sentinel-1 images by date ###
def stitch_sentinel1(date):
    date = ee.Date(date)
    sentinel1_images = ee.ImageCollection("COPERNICUS/S1_GRD").filterBounds(SA) \
        .filterDate(date, date) \
        .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')) \
        .filter(ee.Filter.eq('instrumentMode', 'IW')) \
        .select("VV")

    return ee.Algorithms.If(sentinel1_images.size().gt(0),
                      sentinel1_images.qualityMosaic("system:time_start").set("date", date.format("YYYY-MM-dd")),
                      ee.Image.constant(0).set("date", date.format("YYYY-MM-dd"))) # Return a placeholder if no images
def stitch_sentinel2(date):
    date = ee.Date(date)
    sentinel2_images = ee.ImageCollection("COPERNICUS/S2").filterBounds(SA) \
        .filterDate(date, date) \
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10)) \
        .select(["B8", "B11", "B12"])

    return ee.Algorithms.If(sentinel2_images.size().gt(0),
                      sentinel2_images.qualityMosaic("system:time_start").set("date", date.format("YYYY-MM-dd")),
                      ee.Image.constant(0).set("date", date.format("YYYY-MM-dd"))) # Return a placeholder if no images
# Extract available dates
sentinel1_dates = ee.List(ee.ImageCollection("COPERNICUS/S1_GRD").filterBounds(SA)
    .filterDate(start_date, end_date)
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
    .filter(ee.Filter.eq('instrumentMode', 'IW'))
    .aggregate_array('system:time_start')).map(ee.Date)

sentinel2_dates = ee.List(ee.ImageCollection("COPERNICUS/S2").filterBounds(SA)
    .filterDate(start_date, end_date)
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10))
    .aggregate_array('system:time_start')).map(ee.Date)
sentinel1_check = ee.ImageCollection("COPERNICUS/S1_GRD").filterBounds(SA).filterDate(start_date, end_date).size().getInfo()
sentinel2_check = ee.ImageCollection("COPERNICUS/S2").filterBounds(SA).filterDate(start_date, end_date).size().getInfo()
print("Sentinel-1 Image Count:", sentinel1_check)
print("Sentinel-2 Image Count:", sentinel2_check)

# Find matching dates between Sentinel-1 and Sentinel-2
formatted_sentinel1_dates = sentinel1_dates.map(lambda date: ee.Date(date).format("YYYY-MM-dd"))
formatted_sentinel2_dates = sentinel2_dates.map(lambda date: ee.Date(date).format("YYYY-MM-dd"))

common_dates = formatted_sentinel1_dates.filter(ee.Filter.inList('item', formatted_sentinel2_dates))
#common_dates = sentinel1_dates.filter(ee.Filter.inList('system:time_start', sentinel2_dates))
print("Number of matching dates:", common_dates.size().getInfo())

# Create stitched collections
if common_dates.size().getInfo() > 0:
    stitched_sentinel1 = ee.ImageCollection(common_dates.map(stitch_sentinel1)).filter(ee.Filter.neq("VV", 0))
    stitched_sentinel2 = ee.ImageCollection(common_dates.map(stitch_sentinel2)).filter(ee.Filter.neq("B8", 0))
else:
    print("No matching dates found. Adjust date range or check availability.")

#print("Stitched Sentinel-1 Size:", stitched_sentinel1.size().getInfo())
#print("Stitched Sentinel-2 Size:", stitched_sentinel2.size().getInfo())


### Step 3: Combine Sentinel-1 & Sentinel-2 stitched images and classify ###
# Add a check to see if the stitched collections are empty after filtering placeholders
if stitched_sentinel1.size().getInfo() == 0 or stitched_sentinel2.size().getInfo() == 0:
     print("Stitched collections are empty after filtering out placeholder images.")
else:
    print("Stitched Sentinel-1 Size:", stitched_sentinel1.size().getInfo())
    print("Stitched Sentinel-2 Size:", stitched_sentinel2.size().getInfo())

    ### Step 3: Combine Sentinel-1 & Sentinel-2 stitched images and classify ###
    def classify_image(date):
        date = ee.Date(date)

        # Load stitched Sentinel-1 and Sentinel-2 images
        # Use first() on the filtered collections as they should contain only one image per date
        sentinel2 = stitched_sentinel2.filter(ee.Filter.eq("date", date.format("YYYY-MM-dd"))).first()
        sentinel1 = stitched_sentinel1.filter(ee.Filter.eq("date", date.format("YYYY-MM-dd"))).first()

        # Ensure images exist before merging
        # This check using getInfo() will fail if sentinel1 or sentinel2 is None/null EE object
        # Instead, rely on the fact that we filtered out empty dates and placeholder images
        # from the stitched collections before mapping.
        # We can add a check within the EE function itself using ee.Algorithms.If if needed
        # but given the filtering done previously, it might be redundant.

        # Merge Sentinel-1 and Sentinel-2 bands
        # Need to check if the first() operation resulted in a valid image (not null)
        combined = ee.Algorithms.If(
            ee.Image(sentinel2).bandNames().contains("B8").And(ee.Image(sentinel1).bandNames().contains("VV")),
            ee.Image(sentinel2).addBands(sentinel1),
            ee.Image.constant(-9999).rename("classification") # Return a dummy image if merge fails
        )

        # Check if combined is the dummy image before proceeding with classification
        classified = ee.Algorithms.If(
            ee.Image(combined).bandNames().contains("classification").And(ee.Image(combined).get("constant") == -9999),
            ee.Image(combined), # Return the dummy image if merge failed
            # Otherwise, proceed with classification
            ee.Algorithms.If(
                 archaeology_points.size().gt(0), # Ensure training points exist
                 ee.Image(combined).classify(ee.Classifier.smileRandomForest(10).train(ee.Image(combined).sampleRegions(collection=archaeology_points, properties=['class'], scale=30), 'class')),
                 ee.Image.constant(-9999).rename("classification") # Return a dummy image if no training points
            )
        )

        return ee.Image(classified).set("date", date.format("YYYY-MM-dd"))  # Store date


    # Apply classification
    # Filter out the dummy images
    classified_images = ee.ImageCollection(common_dates.map(classify_image)).filter(ee.Filter.neq("classification", -9999))

    # Create Map
    m = geemap.Map(center=[-14.2, -51.9], zoom=4)

    # Display Sentinel-1 stitched images (only if the collection is not empty)
    if stitched_sentinel1.size().getInfo() > 0:
        list_s1 = stitched_sentinel1.toList(stitched_sentinel1.size())
        for i in range(list_s1.size().getInfo()):
            img = ee.Image(list_s1.get(i))
            date_label = img.get("date").getInfo()
            vis_params_s1 = {"bands": ["VV"], "min": -20, "max": 0, "gamma": 1.4}
            m.addLayer(img, vis_params_s1, f"Sentinel-1 Stitched ({date_label})")

    # Display Sentinel-2 stitched images (only if the collection is not empty)
    if stitched_sentinel2.size().getInfo() > 0:
        list_s2 = stitched_sentinel2.toList(stitched_sentinel2.size())
        for i in range(list_s2.size().getInfo()):
            img = ee.Image(list_s2.get(i))
            date_label = img.get("date").getInfo()
            vis_params_s2 = {"bands": ["B12", "B11", "B8"], "min": 0, "max": 3000, "gamma": 1.4}
            m.addLayer(img, vis_params_s2, f"Sentinel-2 Stitched ({date_label})")

    # Display Classified Images (only if the collection is not empty)
    if classified_images.size().getInfo() > 0:
        list_classified = classified_images.toList(classified_images.size())
        for i in range(list_classified.size().getInfo()):
            img = ee.Image(list_classified.get(i))
            date_label = img.get("date").getInfo()
            vis_params_classified = {
                "bands": ["classification"],  # Assuming 'classification' is the output band
                "min": 0,
                "max": 1,
                "palette": ["red", "yellow", "green"] # Adjust palette based on your classification classes
            }
            m.addLayer(img, vis_params_classified, f"Classified Archaeological Predictions ({date_label})")
    else:
        print("No classified images to display.")

    m.addLayer(mask, {"color": "black"}, "South America Mask")
    m


In [ ]:
# Create Map
m = geemap.Map(center=[-14.2, -51.9], zoom=4)

# Display Sentinel-1 images for common dates
list_s1 = common_dates.map(lambda date: ee.ImageCollection("COPERNICUS/S1_GRD").filterBounds(SA)
                           .filterDate(date, date)
                           .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
                           .filter(ee.Filter.eq('instrumentMode', 'IW'))
                           .select("VV")
                           .qualityMosaic("system:time_start").set("date", date))

for i in range(list_s1.size().getInfo()):
    img = ee.Image(list_s1.get(i))
    date_label = img.get("date").getInfo()
    vis_params_s1 = {"bands": ["VV"], "min": -20, "max": 0, "gamma": 1.4}
    m.addLayer(img, vis_params_s1, f"Sentinel-1 ({date_label})")

# Display Sentinel-2 images for common dates
list_s2 = common_dates.map(lambda date: ee.ImageCollection("COPERNICUS/S2").filterBounds(SA)
                           .filterDate(date, date)
                           #.filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10))
                           .select(["B8", "B11", "B12"])
                           .qualityMosaic("system:time_start").set("date", date))

for i in range(list_s2.size().getInfo()):
    img = ee.Image(list_s2.get(i))
    date_label = img.get("date").getInfo()
    vis_params_s2 = {"bands": ["B12", "B11", "B8"], "min": 0, "max": 3000, "gamma": 1.4}
    m.addLayer(img, vis_params_s2, f"Sentinel-2 ({date_label})")

m.addLayer(mask, {"color": "black"}, "South America Mask")
m


In [ ]:
import geemap
# Define region of interest
m = geemap.Map(center=[-14.2, -51.9], zoom=4)
mask = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017').filter(ee.Filter.eq('country_na', 'Brazil'))
SA = mask.geometry()
m.addLayer(mask, {"color": "black"}, "South America Mask")
m

# Load archaeological points
archaeology_points = ee.FeatureCollection('users/mohamadawadlebanon/Archeologicalsites')

# Define date range
start_date = '2024-03-01'
end_date = '2024-04-30'

# Extract available dates for Sentinel-1
sentinel1_dates = ee.List(ee.ImageCollection("COPERNICUS/S1_GRD").filterBounds(SA)
    .filterDate(start_date, end_date)
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
    .filter(ee.Filter.eq('instrumentMode', 'IW'))
    .aggregate_array('system:time_start')).map(ee.Date)

# Extract available dates for Sentinel-2 (with cloud cover < 10%)
sentinel2_dates = ee.List(ee.ImageCollection("COPERNICUS/S2").filterBounds(SA)
    .filterDate(start_date, end_date)
    #.filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 50))
    .aggregate_array('system:time_start')).map(ee.Date)

# Find exact matching dates
#common_dates = sentinel1_dates.filter(lambda date: sentinel2_dates.contains(date))
#common_dates = sentinel1_dates.map(lambda date: ee.Algorithms.If(sentinel2_dates.contains(date), date, None)) \
#    .removeAll([None])
common_dates = sentinel1_dates.filter(ee.Filter.inList('item', sentinel2_dates))
#print(common_dates.size().getInfo())  # Should be > 0
formatted_dates = sentinel1_dates.map(lambda date: ee.Date(date).format("YYYY-MM-dd"))
print("Sentinel-1 Dates:", formatted_dates.getInfo())
formatted_dates = sentinel2_dates.map(lambda date: ee.Date(date).format("YYYY-MM-dd"))
print("Sentinel-2 Dates:", formatted_dates.getInfo())



# Function to combine and classify images by matching date
def classify_image(date):
    date = ee.Date(date)

    # Load matching Sentinel-1 and Sentinel-2 images
    sentinel2 = ee.ImageCollection("COPERNICUS/S2").filterBounds(SA) \
        .filterDate(date, date) \
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10)) \
        .select(["B8", "B11", "B12"]).median()

    sentinel1 = ee.ImageCollection("COPERNICUS/S1_GRD").filterBounds(SA) \
        .filterDate(date, date) \
        .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')) \
        .filter(ee.Filter.eq('instrumentMode', 'IW')) \
        .select("VV").median()

    combined = sentinel2.addBands(sentinel1)

    # Train classifier using archaeological points
    training = combined.sampleRegions(collection=archaeology_points, properties=['class'], scale=30)
    classifier = ee.Classifier.smileRandomForest(10).train(training, 'class')

    # Classify image
    classified = combined.classify(classifier)

    return classified.set("date", date.format("YYYY-MM-dd"))  # Store date

# Apply classification to each image in the collection
classified_images = ee.ImageCollection(common_dates.map(classify_image))

# Create Map
m = geemap.Map(center=[-14.2, -51.9], zoom=4)

# Display each classified image separately
list_of_images = classified_images.toList(classified_images.size())

for i in range(list_of_images.size().getInfo()):
    img = ee.Image(list_of_images.get(i))
    date_label = img.get("date").getInfo()

    vis_params = {
        "bands": ["B12", "B11", "VV"],
        "min": 0,
        "max": 1,
        "gamma": 1.4
    }

    m.addLayer(img, vis_params, f"Archaeological Predictions ({date_label})")

m.addLayer(mask, {"color": "black"}, "South America Mask")
m


In [ ]:
import geemap
import ee

# Initialize Earth Engine
ee.Initialize(project='velvety-ring-328419')
m = geemap.Map(center=[-14.2, -51.9], zoom=4)
# Define region of interest (South America)
mask = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017').filter(ee.Filter.eq('wld_rgn', 'South America'))
SA = mask.geometry()

# Load MODIS NDVI Data
modis_col = ee.ImageCollection('MODIS/006/MOD13A2').select('NDVI').filterBounds(SA).filterDate('2024-01-01', '2024-12-31')
modis_mosaic = modis_col.mosaic().unitScale(0, 10000)  # Normalize NDVI

# Load Sentinel-2 Data (NIR & SWIR Bands)
sentinel2_col = ee.ImageCollection("COPERNICUS/S2").filterBounds(SA).filterDate('2024-01-01', '2024-12-31') \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 50)).select(["B8", "B11", "B12"])  # Select NIR & SWIR bands
sentinel2_mosaic = sentinel2_col.mosaic().divide(10000)  # Normalize reflectance

# Load Sentinel-1 Data (SAR VV)
sentinel1_col = ee.ImageCollection("COPERNICUS/S1_GRD").filterBounds(SA).filterDate('2024-01-01', '2024-12-31') \
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')).filter(ee.Filter.eq('instrumentMode', 'IW')) \
    .select("VV")
sentinel1_mosaic = sentinel1_col.mosaic().unitScale(-25, 0)  # Convert backscatter to dB

# Combine Datasets into One Composite Image
combined = sentinel2_mosaic.addBands(sentinel1_mosaic).addBands(modis_mosaic)

# Visualization Parameters
vis_params = {
    "bands": ["B12", "B11", "VV"],  # SWIR & SAR for archaeology visibility
    "min": 0,
    "max": 1,
    "gamma": 1.4
}

# Create Map
m = geemap.Map(center=[-14.2, -51.9], zoom=4)
m.addLayer(combined, vis_params, "Sentinel-2 + Sentinel-1 + NDVI Composite")
m.addLayer(mask, {"color": "black"}, "South America Mask")
m


In [ ]:
!pip install /content/ee-packages-py-main/

In [ ]:

import geemap


# Initialize Earth Engine (assuming Authenticate() has been run previously)
ee.Initialize(project='velvety-ring-328419')

# Define region of interest (South America)
mask = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017').filter(ee.Filter.eq('wld_rgn', 'South America'))
SA = mask.geometry()

# Load MODIS NDVI data and mosaic
modis_col = ee.ImageCollection('MODIS/006/MOD13A2').select('NDVI').filterBounds(SA).filterDate('2024-01-01', '2024-12-31')
# Create a mosaic of the MODIS collection for visualization
modis_mosaic = modis_col.mosaic()

# Load Sentinel-2 (Optical Data) and mosaic
# Filter by bounds before filtering by date to be more efficient
sentinel2_col = ee.ImageCollection("COPERNICUS/S2").filterBounds(SA).filterDate('2024-01-01', '2024-12-31').filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 50)) # Add cloud filter
# Create a mosaic of the Sentinel-2 collection
sentinel2_mosaic = sentinel2_col.mosaic()


# Load Sentinel-1 (SAR Data) and mosaic
# Filter by bounds before filtering by date to be more efficient
# Changed the collection ID from 'COPERNICUS/S1' to 'COPERNICUS/S1_GRD'
sentinel1_col = ee.ImageCollection("COPERNICUS/S1_GRD").filterBounds(SA).filterDate('2024-01-01', '2024-12-31').filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')).filter(ee.Filter.eq('instrumentMode', 'IW')) # Filter for a specific mode
# Create a mosaic of the Sentinel-1 collection
sentinel1_mosaic = sentinel1_col.mosaic()


# Visualization parameters
modis_vis = {
    'min': 0,
    'max': 8000,
    'palette': ['red', 'yellow', 'green']  # Single-band, palette allowed
}
sentinel2_vis = {
    'bands': ['B4', 'B3', 'B2'],  # True color visualization
    'min': 0,
    'max': 3000
}
sentinel1_vis = {
    'min': -25,
    'max': 0
}

# Create map
m = geemap.Map(center=[-14.2, -51.9], zoom=4)

# Add the mosaic images to the map
m.addLayer(modis_mosaic, modis_vis, "MODIS NDVI Mosaic")
m.addLayer(sentinel2_mosaic, sentinel2_vis, "Sentinel-2 Mosaic (True Color)")
m.addLayer(sentinel1_mosaic, sentinel1_vis, "Sentinel-1 Mosaic (VV)")


# Add region mask
m.addLayer(mask, {'color': 'black'}, "South America Mask")

# Display the map
m

In [ ]:
#print(modis_col.size().getInfo())  # Should return >0 if images exist
print(sentinel2_col.size().getInfo())
#print(sentinel1_col.size().getInfo())


In [ ]:
import geemap
import ee

# Initialize Earth Engine
ee.Initialize(project='velvety-ring-328419')

# Define region of interest (South America)
mask = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017').filter(ee.Filter.eq('wld_rgn', 'South America'))
SA = mask.geometry()

# Function to group images by month and stitch them
def stitch_images_by_month(image_collection, year, month):
    filtered_col = image_collection.filter(ee.Filter.calendarRange(year, year, 'year')) \
                                   .filter(ee.Filter.calendarRange(month, month, 'month'))
    return filtered_col.mosaic().set('year', year, 'month', month)

# Load MODIS NDVI data and process by month
modis_col = ee.ImageCollection('MODIS/006/MOD13A2').select('NDVI').filterBounds(SA).filterDate('2023-04-01', '2023-08-31')
modis_stitched = stitch_images_by_month(modis_col, 2023, 6)  # Example for June

# Load Sentinel-2 data and process by month
sentinel2_col = ee.ImageCollection("COPERNICUS/S2").filterBounds(SA).filterDate('2023-04-01', '2023-08-31').filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))
sentinel2_stitched = stitch_images_by_month(sentinel2_col, 2023, 6)

# Load Sentinel-1 data and process by month
sentinel1_col = ee.ImageCollection("COPERNICUS/S1_GRD").filterBounds(SA).filterDate('2023-04-01', '2023-08-31').filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')).filter(ee.Filter.eq('instrumentMode', 'IW'))
sentinel1_stitched = stitch_images_by_month(sentinel1_col, 2023, 6)

# Visualization parameters
modis_vis = {
    'min': 0,
    'max': 8000,
    'palette': ['red', 'yellow', 'green']
}
sentinel2_vis = {
    'bands': ['B4', 'B3', 'B2'],
    'min': 0,
    'max': 3000
}
sentinel1_vis = {
    'min': -25,
    'max': 0
}

# Create map
m = geemap.Map(center=[-14.2, -51.9], zoom=4)

# Add stitched images to the map
m.addLayer(modis_stitched, modis_vis, "MODIS NDVI Stitched by Month")
m.addLayer(sentinel2_stitched, sentinel2_vis, "Sentinel-2 Stitched by Month")
m.addLayer(sentinel1_stitched, sentinel1_vis, "Sentinel-1 Stitched by Month")

# Add region mask
m.addLayer(mask, {'color': 'black'}, "South America Mask")

# Display the map
m


In [ ]:
#text = require('users/gena/packages:text');
#style = require('users/gena/packages:style');
#utils = require('users/gena/packages:utils');
min = -1;
max = 1;
textProperties = {
  'fontSize': 16,
  'textColor': 'red',
  'outlineColor': 'black',
  'outlineWidth': 0.1,
  'outlineOpacity': 0.6
};
labels = ee.List.sequence(min, max);
gradientBar = style.GradientBar.draw(geometry3, {
  'min': min, 'max': max, 'palette': visParams.palette, 'labels': labels,
  'format': '%.0f', 'text': textProperties});